In [2]:
from bs4 import BeautifulSoup
import requests
import re
import urllib.parse

#조선일보 
def chosun_newsurl(keyword, date1, date2, pagenum):
    url=("http://search.chosun.com/search/news.search?query={0}" \
"&pageno={3}&orderby=news&naviarraystr=&kind=&cont1=&cont2=&cont5=&categoryname=&categoryd2=&c_scope=&sdate={1}" \
"&edate={2}" \
"&premium="
    .format(keyword,date1,date2,pagenum))
    resp = requests.get(url)
    resp.raise_for_status()
    resp.encoding = 'utf-8'
    html = resp.text
    soup = BeautifulSoup(html, 'html.parser')
    my_titles = str(soup.select('dt, a'))
    news_url = re.findall('href="(http:.+?)"', my_titles)
    return (news_url)

    
from datetime import timedelta
import datetime, re

date1=datetime.date(2018, 10, 1)
date2=datetime.date(2018, 12, 31)
keyword='유치원3법'

url_list = set()
while date1 < date2:
    print(date1)
    for num in range(20):
        t = chosun_newsurl(keyword, re.sub('-','.',str(date1)), re.sub('-','.',str(date2)), num)
        url = [i for i in t if re.search('2018',i)]
        
        for i in url:
            url_list.add(i)
        date1+=timedelta(days=+1)
    
def chosun_news(url):
    source_code_from_url = urllib.request.urlopen(url)
    soup = BeautifulSoup(source_code_from_url, 'lxml', from_encoding = 'utf-8')
    text = str(soup.select('#news_body_id'))
    burn = re.compile('\t|\n|\r|\'|\[|\]|<.+?>')
    article = burn.sub('',text)
    burn = re.compile('좋아요(.+) ')
    article = burn.sub('',article)
    press = 'chosun'
    p = re.compile('입력(.+?)\|*[가-힣]')
    date = re.findall (p, article)
    new_date = []
    for i in date:
        if re.search('\d\d\d\d\.\d\d\.\d\d',i):
            p = re.compile('(\d\d\d\d\.\d\d\.\d\d)')
            new_date+=p.findall(i)
        else:
            new_date+=i
    article = p.sub('',article)
    article = re.sub('\.','\n',article)
    return (article, press, ''.join(new_date)) 

import pandas as pd 

article = []
press = []
date = []
url = []

for i in url_list:
    if chosun_news(i)[0]!='':
        article.append(chosun_news(i)[0])        
        press.append(chosun_news(i)[1])
        date.append(chosun_news(i)[2])
        url.append(i)
        
new_article=[]
for i in article:
    if re.search('입력.+[가-힣]',i):
        p = re.compile('(입력.+)[가-힣]')
        new_article.append(p.sub('',i))
    else:
        new_article.append(i)
    
data = {'article': new_article, 'press': press, 'date': date,'url': url}
df = pd.DataFrame(data, columns = ['article', 'press', 'date','url'])
df.to_csv(r'C:/Users/chsjk/OneDrive/Documents/학업/4학년 겨울학기/웹스크롤러, 형태소 분석기,코퍼스/article/조선_비트코인.csv')

df[['article', 'press', 'date', 'url']]

2018-01-01
2018-01-21
2018-02-10


,article,press,date,url
0,"\n CNBC에 따르면, 쉴러 교수는 이날 스위스 다보스(Davos)에서 열린 세계...",chosun,2018.01.29,http://news.chosun.com/site/data/html_dir/2018...
1,\n 미국 금융 당국이 본격 조사에 착수한 가운데 미국 뉴욕타임스와 블룸버그 등 주...,chosun,2018.02.02,http://biz.chosun.com/site/data/html_dir/2018/...
2,\n 비트코인은 한때 1만달러선을 내주기도 했다\n 중국 정부의 규제 가능성(개인 ...,chosun,2018.01.18,http://biz.chosun.com/site/data/html_dir/2018/...
3,\n 길거리에도 투자 카페들이 여기저기 눈에 띄죠\n 이 카페는 스타트업(창업 초기...,chosun,2018.02.05,http://biz.chosun.com/site/data/html_dir/2018/...
4,\n 전 세계적으로 한때 패닉셀(공포에 질려서 투매하는 현상)이 벌어지면서 하루 사...,chosun,2018.02.07,http://news.chosun.com/site/data/html_dir/2018...
5,\nCNBC는 이날 “한국의 박상기 법무부 장관이 가상화폐 거래소 폐지 규제안까지 ...,chosun,2018.01.12,http://news.chosun.com/site/data/html_dir/2018...
6,\n 자신이 생산한 포도가 서울의 백화점에서는 자신이 판 가격보다 3배 이상으로 팔...,chosun,2018.01.27,http://news.chosun.com/site/data/html_dir/2018...
7,\n3일 연속 약세를 보이며 전날 한때 660만원까지 떨어졌던 비트코인이 7일 90...,chosun,2018.02.08,http://biz.chosun.com/site/data/html_dir/2018/...
8,"4\n7%, 코스피 1\n5% 떨어져비트코인 한때 660만원, 또 폭락미국 증시 ...",chosun,2018.02.07,http://news.chosun.com/site/data/html_dir/2018...
9,\n”지난 11일 A금융공기업 직원 3명은 사내 엘리베이터에서 가상화폐 투자 금지에...,chosun,2018.01.16,http://biz.chosun.com/site/data/html_dir/2018/...
